In [ ]:
%pip install tf-nightly

In [1]:
import tensorflow as tf
tf.enable_v2_behavior()
import pandas as pd
print(tf.__version__)

inputs = {
    'a' : tf.keras.layers.Input(name='a', shape=(), dtype='float32'),
    'b' : tf.keras.layers.Input(name='b', shape=(), dtype='string'),
}
fc = {
    'a' : tf.feature_column.numeric_column('a'),
    'b' : tf.feature_column.indicator_column(
        tf.feature_column.categorical_column_with_vocabulary_list('b', ['one', 'two'])
    )
}

ds = tf.keras.layers.DenseFeatures(fc.values())(inputs)
output = tf.keras.layers.Dense(1)(ds)
model = tf.keras.models.Model(inputs=inputs, outputs=output)
model.compile(optimizer='adam', loss='mse')

df = pd.DataFrame.from_dict({
    'a': [3.0, 2.0, 1.0],
    'b': ['one', 'one', 'two'],
})
y =  [3.0, 2.0, 2.0]

model.fit([df['a'], df['b']], y, steps_per_epoch=1)

model.predict({
    'a': tf.convert_to_tensor([3.0]),
    'b': tf.convert_to_tensor(['one']),
})

1.14.1-dev20190603


W0604 05:24:37.030460 140714490566400 deprecation.py:323] From /home/jupyter/.local/lib/python3.5/site-packages/tensorflow/python/feature_column/feature_column_v2.py:2655: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0604 05:24:37.060478 140714490566400 deprecation.py:323] From /home/jupyter/.local/lib/python3.5/site-packages/tensorflow/python/feature_column/feature_column_v2.py:4215: IndicatorColumn._variable_shape (from tensorflow.python.feature_column.feature_column_v2) is deprecated and will be removed in a future version.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
W0604 05:24:37.061576 140714490566400 deprecation.py:323] From /home/jupyter/.local/lib/python3.5/site-packages/tensorflow/python/feature_column/feature_column_v

Train on 1 samples
1/1 [==============================] - 0s 74ms/step - loss: 22.2133


array([[-3.2158718]], dtype=float32)

In [7]:
# This doesn't work.
model.save('04_keras/export/simple.h5')
newmodel = tf.keras.models.load_model('04_keras/export/simple.h5')
newmodel.predict({
    'a': tf.convert_to_tensor([3.0]),
    'b': tf.convert_to_tensor(['one']),
})

ValueError: Unknown layer: DenseFeatures

In [3]:
# this should work, but doesn't.
tf.keras.experimental.export_saved_model(model, '04_keras/export/savedmodel')

TypeError: __init__() missing 1 required positional argument: 'feature_columns'